## construct a graph

In [36]:
import os
import sys
import casadi as cd
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))
sys.path.append(os.path.abspath(os.getcwd()))
from Envs.UAV import UAV_env,UAV_model
from Solvers.OCsolver import ocsolver,ocsolver_fast,ocsolver_inner_Barrier
import numpy as np
from matplotlib import pyplot as plt
from Solvers.Cutter import cutter

In [37]:
uav_params={'gravity':10,'m':1,'J_B':np.eye(3),'l_w':0.5,'dt':0.1,'c':1}
uav_env=UAV_env(**uav_params)


uav_model=UAV_model(**uav_params)
dyn_f=uav_model.get_dyn_f()

In [38]:
Horizon=25
u_dim=4
x_dim=13

In [39]:
X_0=cd.SX.sym('X_0' , x_dim)
X_k=X_0
X_list=[] #x1:T
X_xy_pos_list=[]
U_list=[] #u0:T

for k in range(Horizon): #at step k, construct u_k and x_{k+1}
    U_k=cd.SX.sym('U_t_' + str(k), u_dim)
    U_list.append(U_k)
    X_k_1=dyn_f(X_k,U_k)
    X_list.append(X_k_1)
    X_xy_pos_list.append(X_k_1[0:3])
    X_k=X_k_1

Pos_jacobi=cd.jacobian(cd.vertcat(*X_xy_pos_list),cd.vertcat(*U_list))
Pos_Jacobi_Func=cd.Function('position_jacobian',[X_0,cd.vertcat(*U_list)],[Pos_jacobi])


In [40]:
init_r = np.zeros((3,1))
init_v = np.zeros((3,1))
init_q = np.reshape(np.array([1,0,0,0]),(-1,1))
init_q = np.random.uniform(0,1,(4,1))
init_q/=np.linalg.norm(init_q)
#print(Quat_Rot(init_q))
init_w_B = np.zeros((3,1))
init_x=np.concatenate([init_r,init_v,init_q,init_w_B],axis=0)

In [43]:
#ctrl_input_flat=np.random.uniform(0,4,(100,1))
ctrl_input_flat=2.48*np.ones((100,1)) + np.random.normal(0,0.2,(100,1))
p_jacobi=Pos_Jacobi_Func(init_x,ctrl_input_flat)

In [44]:
arr_jac=np.array(p_jacobi)